# Model Selection in Linear Regression with `Python` and `R`

## Index

* [Data-set ](#1)
* * [Loading the data-set in `Python`](#2)
* * [Loading the data-set in `R`](#3)

  <br>
  
* [F-test: test to compare models](#4)
* * [F-test in `Python`](#5)
* * [F-test in `R`](#6)
* * [ANOVA test as an F-test](#7)
* * [Significance test as an F-test](#8)

  <br>



* [Iterative Algorithms](#9) 
  


* * [Metrics](#10) 
* * * [$\widehat{R}^2$](#11) 
* * * [$AIC$](#12) 
* * * [$BIC$](#13)
* * * * [Maximum Likelihood Estimation in the Linear Regression Model](#14) 
* * * * * [$AIC$ and $BIC$ in the Linear Regression Model](#14.1) 
* * * * [$AIC$ in `Python`](#14) 
* * * * [$BIC$ in `Python`](#15)
* * * * [$AIC$ in `R`](#16) 
* * * * [$BIC$ in `R`](#17)
* * * [Mallows's $C_p$](#18) 
* * * * [$C_p$ in `Python`](#19)
* * * * [$C_p$ in `R`](#19)




* * [Best Subset Selection](#20) 
* * * [Best Subset Selection in `Python`](#21) 
* * * [Best Subset Selection in `R`](#22) 
  


* * [Forward Selection](#23) 
* * * [Forward Selection in `Python`](#24)
* * * [Forward Selection in `R`](#25)
  


* * [Backward Selection](#26) 
* * * [Backward Selection in `Python`](#27) 
* * * [Backward Selection in `R`](#28) 


  


## Data-set <a class="anchor" id="1"></a>

The description of the data-set that we are going to use could be found in the following article:

https://fabioscielzoortiz.github.io/Estadistica4all.github.io/Articulos/Linear%20Regression%20in%20Python%20and%20R.html

### Loading the data-set in `Python` <a class="anchor" id="2"></a>

In [230]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [231]:
data_Python = pd.read_csv('data_Python_copy.csv')

In [232]:
data_Python

,price,size_in_m_2,longitude,latitude,no_of_bedrooms,no_of_bathrooms,quality
0,2700000,100.242337,55.138932,25.113208,1,2,1
1,2850000,146.972546,55.151201,25.106809,2,2,1
2,1150000,181.253753,55.137728,25.063302,3,5,1
3,2850000,187.664060,55.341761,25.227295,2,3,0
4,1729200,47.101821,55.139764,25.114275,0,1,1
...,...,...,...,...,...,...,...
1900,1500000,100.985561,55.310712,25.176892,2,2,3
1901,1230000,70.606280,55.276684,25.166145,1,2,1
1902,2900000,179.302790,55.345056,25.206500,3,5,1
1903,675000,68.748220,55.229844,25.073858,1,2,1


In [233]:
data_Python.dtypes

price                int64
size_in_m_2        float64
longitude          float64
latitude           float64
no_of_bedrooms       int64
no_of_bathrooms      int64
quality              int64
dtype: object

Converting $quality$ variable to categorical:

In [234]:
data_Python['quality'] = data_Python['quality'].astype('category')

### Loading the data-set in `R` <a class="anchor" id="3"></a>

In [235]:
import rpy2

%load_ext rpy2.ipython

import rpy2.robjects as robjects

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [236]:
%%R 

library(tidyverse)

data_R = read_csv('data_Python_copy.csv')

Rows: 1905 Columns: 7
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (7): price, size_in_m_2, longitude, latitude, no_of_bedrooms, no_of_bath...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Converting $quality$ variable to categorical:

In [237]:
%%R
data_R$quality <- as.factor(data_R$quality)

In [238]:
%%R

data_R[1:7, ]

# A tibble: 7 x 7
    price size_in_m_2 longitude latitude no_of_bedrooms no_of_bathrooms quality
    <dbl>       <dbl>     <dbl>    <dbl>          <dbl>           <dbl> <fct>  
1 2700000       100.       55.1     25.1              1               2 1      
2 2850000       147.       55.2     25.1              2               2 1      
3 1150000       181.       55.1     25.1              3               5 1      
4 2850000       188.       55.3     25.2              2               3 0      
5 1729200        47.1      55.1     25.1              0               1 1      
6 3119900        94.3      55.1     25.1              1               2 1      
7 8503600       192.       55.1     25.1              2               3 2      


## F-test: test to compare models <a class="anchor" id="4"></a>


The ANOVA and significance test are a particular case of a more general test that is useful to compere linear regression models, but always under the assumptions of the model.



-   We have a linear regression model with $\hspace{0.1cm} k \hspace{0.1cm} $ coefficients (betas)
    $\hspace{0.1cm} \Rightarrow \hspace{0.1cm} \Omega_k$

     -  $\Omega_k \ : \  \ y_i = \beta_0 + \beta_1\cdot x_{i1} +...+ \beta_{k-1}\cdot x_{i(k-1)} + \varepsilon_i$ 




-   We have another linear regression model with only
    $\hspace{0.1cm} q<k \hspace{0.1cm} $ coefficients (betas) of $\Omega_p$ $\hspace{0.1cm} \Rightarrow \hspace{0.1cm}  \omega_q$

     -  $\omega_q \ : \  \ y_i = \beta_0 + \beta_1\cdot x_{i1} +...+ \beta_{q-1}\cdot x_{i(q-1)} + \varepsilon_i \hspace{0.2cm}$ , with $q < k$





-   $\omega_q$ is a sub-model of $\Omega_k$ , we can denote this as
    $\hspace{0.1cm} \omega_q \subset \Omega_k$
    




The hypothesis test we want to carry out is the following:

$$
H_0: \hspace{0.15cm} \omega_q  \\

H_1: \hspace{0.15cm} \Omega_k
$$


Where :

- $\omega_q \subset \Omega_p$

- Reject $H_0$ means  $\Omega_k$ is a "better" model than $\omega_q$

- Not Reject $H_0$ means $\Omega_k$ isn´t a "better" model than $\omega_q$

<br>



Now we have to determinate a rule to reject $H_0$ in favor of $H_1$ or not

A first approach is the following:

Let :

$$
RSS_{\Omega_k} =  \sum_{i=1}^n ( \hat{\varepsilon}_{\Omega_k \hspace{0.05cm},\hspace{0.05cm} i} )^2 =  \sum_{i=1}^n \left( y_i - \hat{y}_{\hspace{0.01cm} \Omega_k \hspace{0.05cm},\hspace{0.02cm} i}\right)^2   
$$

$$
RSS_{\omega_q} =  \sum_{i=1}^n ( \hat{\varepsilon}_{\omega_q \hspace{0.05cm},\hspace{0.05cm} i} )^2 =  \sum_{i=1}^n \left( y_i - \hat{y}_{\omega_q \hspace{0.05cm},\hspace{0.02cm} i}\right)^2
$$



-   If $\hspace{0.1cm} RSS_{\omega_q} - RSS_{\Omega_k} \hspace{0.1cm}$ is **small**, then the
    predictions of the **smaller model** $(\omega_q)$ are almost as good as the **larger
    model** $(\Omega_k)$, so we would prefer the smaller model on the grounds of
    simplicity.

-   If $\hspace{0.1cm} RSS_{\omega_q} - RSS_{\Omega_k} \hspace{0.1cm}$ is **large**, then the
    predictions of the **smaller model** $(\omega_q)$  are much worse than the **larger
    model** $(\Omega_k)$, so we would prefer the larger model.




That suggest that something like

$$
\dfrac{RSS_{\omega_q} - RSS_{\Omega_k}}{RSS_{\Omega_k}}
$$

would be a potentially good test statistic, where the denominator is used for scaling purposes.





### Statistic Test

Finally we can get to an statistic test based on the previous
expression, called **F-statistic**:




$$
F=\dfrac{(RSS_{\omega_q} - RSS_{\Omega_k})/(k-q)}{RSS_{\Omega_k}/(n-k)} \sim F_{\hspace{0.05cm} k-q \hspace{0.05cm},\hspace{0.05cm} n-k}
$$


If $\hspace{0.1cm} df_{\Omega_k} = n-k \hspace{0.1cm}$ and $\hspace{0.1cm} df_{\omega_q} = n-q \hspace{0.1cm}$ , then :



$$
F=\dfrac{(RSS_{\omega_q} - RSS_{\Omega_k})/(df_{\omega_q} - df_{\Omega_k})}{RSS_{\Omega_k}/df_{\Omega_k}} \sim F_{\hspace{0.05cm} (df_{\omega_q} - df_{\Omega_k}) \hspace{0.05cm},\hspace{0.05cm} df_{\Omega_k}}
$$


Where:

$k$ is the number of coefficients (betas) of the model $\Omega_k$

$q$ is the number of coefficients (betas) of the model $\omega_q$


The beauty of this approach is you only need to know the general form.
In any particular case, you just need to figure out which model
represent the null and alternative hypothesis, fit them and compute the test statistic.




### F-test in `Python` <a class="anchor" id="5"></a>

In [239]:
import statsmodels.formula.api as smf

from statsmodels.stats.anova import anova_lm

from statsmodels.formula.api import ols

In [240]:
M1_py = smf.ols(formula = 'price ~ size_in_m_2*quality  + no_of_bedrooms + no_of_bathrooms + latitude + longitude', 
                 data =data_Python).fit()

M2_py = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude ', 
                 data =data_Python).fit()

M3_py = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + latitude + longitude', 
                 data =data_Python).fit()

M4_py = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms', 
                 data =data_Python).fit()

M5_py = smf.ols(formula = 'price ~ size_in_m_2', 
                 data =data_Python).fit()

$M1$:$\hspace{0.15cm}$ price ~ size_in_m_2*quality  + no_of_bedrooms + no_of_bathrooms + latitude + longitude

$M2$:$\hspace{0.15cm}$  price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude 

$M3$:$\hspace{0.15cm}$  price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + latitude + longitude

$M4$:$\hspace{0.15cm}$  price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms

$M5$:$\hspace{0.15cm}$  price ~ size_in_m_2



Note that:

$$
M5 \subset M4 \subset M3 \subset M2 \subset M1
$$


We are going to use the function `anova_lm` to carry out the F-test in `Python`

The usage of anova_lm is pretty simple:

> anova_lm(Model $H_0$ , Model $H_1$)
>
> $\hspace{0.25cm}$ Where : $\hspace{0.15cm}$ Model $H_0$ $\subset$ Model $H_1$



Remember that the hypothesis test that we want to do is: 

$$
H_0: \hspace{0.15cm} \omega_q \hspace{0.1cm} (Model \hspace{0.1cm} H_0) \\ 
H_1: \hspace{0.15cm} \Omega_k \hspace{0.1cm} (Model \hspace{0.1cm} H_1)
$$


For example, to carry out the hypothesis test

$$
H_0: \hspace{0.15cm} M2\_py  \hspace{0.15cm} (\omega_q)  \\ 
H_1: \hspace{0.15cm} M1\_py  \hspace{0.15cm} (\Omega_q)
$$

where M2\_py is the **smallest** model and M1\_py the **largest**, we use the following code:

In [241]:
anova_lm(M2_py, M1_py)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,1896.0,4.882082e+15,0.0,NaN,NaN,NaN
1,1893.0,4.694567e+15,3.0,1.875147e+14,25.203983,5.483226e-16


For example, this output gives to us the following information:


$$
F=\dfrac{(RSS_{M2\_{py}} - RSS_{M1\_py})/(df_{\omega_q} - df_{\Omega_k})}{RSS_{M1\_{py}}/ df_{\Omega_k}} = 25.203983
$$ 

And the p-value of the test is:

$$
P(F_{\hspace{0.05cm} (df_{\omega_q} - df_{\Omega_k}) \hspace{0.05cm},\hspace{0.05cm} df_{\Omega_k}} > 25.203983) = 5.483226e-16
$$

So, the conclusion for any $\hspace{0.1cm} \alpha > 5.483226e-16 \hspace{0.1cm}$ is **reject** $\hspace{0.1cm} H_0: M2\_py\hspace{0.1cm}$ in favor of $\hspace{0.1cm}H_1: M1\_py\hspace{0.1cm}$, so we can accept that $\hspace{0.1cm} M1\_py \hspace{0.1cm}$ is a **better** model than $\hspace{0.1cm} M2\_py \hspace{0.1cm}$

Let's to check that ;

First of all, note that the number of betas of model M1_py is $k=12$ (because $quality$ has four categories and there are an interaction between $quality$ and $size\_in\_m\_2$ , and for M2_py is $q=9$

This can be easily seen in the following outputs given by `print(M1_py.summary())` and `print(M2_py.summary())`



In the other hand: 

$df_{M1\_py} = n - k = 1905 - 12 = 1893 =$ `anova_lm(M2_py, M1_py).df_resid[1]`

$df_{M2\_py} = n - q = 1905 - 9 = 1896 =$ `anova_lm(M2_py, M1_py).df_resid[0]`

$RSS_{M1\_py} =$ `anova_lm(M2_py, M1_py).ssr[1]` $= 4.694567e+15$	 

$RSS_{M2\_py} =$ `anova_lm(M2_py, M1_py).ssr[0]` $= 4.882082e+15$	



The following step is to compute the F-statistic  using the above information :

In [242]:
numerator = ( anova_lm(M2_py, M1_py).ssr[0] - anova_lm(M2_py, M1_py).ssr[1] ) / ( anova_lm(M2_py, M1_py).df_resid[0] - anova_lm(M2_py, M1_py).df_resid[1] )

denominator = anova_lm(M2_py, M1_py).ssr[1] / anova_lm(M2_py, M1_py).df_resid[1]

F = numerator / denominator

In [243]:
F

25.203983015834208

In [244]:
import scipy

In [245]:
from scipy.stats import f, norm

In [246]:
pvalor = 1 - scipy.stats.f.cdf(F, anova_lm(M2_py, M1_py).df_resid[0] - anova_lm(M2_py, M1_py).df_resid[1] , anova_lm(M2_py, M1_py).df_resid[1]  )

In [247]:
pvalor

5.551115123125783e-16

If you have any doubt about k=12 and q=9 look the following outputs:

In [248]:
print(M1_py.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     420.2
Date:               ma., 06 sep. 2022   Prob (F-statistic):               0.00
Time:                        11:34:41   Log-Likelihood:                -29881.
No. Observations:                1905   AIC:                         5.979e+04
Df Residuals:                    1893   BIC:                         5.985e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

This output doesn´t show longitude variable, because the output exceeds the size limit. So if you count all the parameters (betas) the result will be  k=12

In [249]:
print(M2_py.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     547.4
Date:               ma., 06 sep. 2022   Prob (F-statistic):               0.00
Time:                        11:34:43   Log-Likelihood:                -29918.
No. Observations:                1905   AIC:                         5.985e+04
Df Residuals:                    1896   BIC:                         5.990e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -6.207e+07   2.99e+07     

This output contain all the parameters of the model inside, if you count you will have q=9

In [250]:
anova_lm(M3_py, M2_py)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,1899.0,4.898667e+15,0.0,NaN,NaN,NaN
1,1896.0,4.882082e+15,3.0,1.658498e+13,2.146975,0.092395


This output gives to us the following information:


$$
F=\dfrac{(RSS_{M2\_{py}} - RSS_{M1\_py})/(df_{\omega_q} - df_{\Omega_k})}{RSS_{M1\_{py}}/ df_{\Omega_k}} = 2.146975
$$ 

And the p-value of the test is:

$$
P(F_{\hspace{0.05cm} (df_{\omega_q} - df_{\Omega_k}) \hspace{0.05cm},\hspace{0.05cm} df_{\Omega_k}} > 25.203983) = 0.092395
$$

So, the conclusion for the habitual $\hspace{0.1cm}\alpha = 0.01 \hspace{0.1cm}or\hspace{0.1cm} 0.05\hspace{0.1cm} $
 , is **not reject** $\hspace{0.1cm} H_0: M3\_py\hspace{0.1cm}$ in favor of $\hspace{0.1cm}H_1: M2\_py\hspace{0.1cm}$, so we can accept that $\hspace{0.1cm} M3\_py \hspace{0.1cm}$ is a **better** model than $\hspace{0.1cm} M2\_py \hspace{0.1cm}$

In [251]:
anova_lm(M4_py, M3_py)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,1901.0,5.080302e+15,0.0,NaN,NaN,NaN
1,1899.0,4.898667e+15,2.0,1.816345e+14,35.205892,9.703235e-16


This output gives to us the following information:


$$
F=\dfrac{(RSS_{M2\_{py}} - RSS_{M1\_py})/(df_{\omega_q} - df_{\Omega_k})}{RSS_{M1\_{py}}/ df_{\Omega_k}} = 35.205892
$$ 

And the p-value of the test is:

$$
P(F_{\hspace{0.05cm} (df_{\omega_q} - df_{\Omega_k}) \hspace{0.05cm},\hspace{0.05cm} df_{\Omega_k}} > 25.203983) = 9.703235e-16

$$

So, the conclusion for any $\hspace{0.1cm} \alpha > 9.703235e-16 \hspace{0.1cm}$ is **reject** $\hspace{0.1cm} H_0: M4\_py\hspace{0.1cm}$ in favor of $\hspace{0.1cm}H_1: M3\_py\hspace{0.1cm}$, so we can accept that $\hspace{0.1cm} M3\_py \hspace{0.1cm}$ is a **better** model than $\hspace{0.1cm} M4\_py \hspace{0.1cm}$

In [252]:
anova_lm(M5_py, M1_py)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,1903.0,5.593743e+15,0.0,NaN,NaN,NaN
1,1893.0,4.694567e+15,10.0,8.991756e+14,36.257642,2.127326e-65


This output gives to us the following information:


$$
F=\dfrac{(RSS_{M2\_{py}} - RSS_{M1\_py})/(df_{\omega_q} - df_{\Omega_k})}{RSS_{M1\_{py}}/ df_{\Omega_k}} = 36.257642
$$ 

And the p-value of the test is:

$$
P(F_{\hspace{0.05cm} (df_{\omega_q} - df_{\Omega_k}) \hspace{0.05cm},\hspace{0.05cm} df_{\Omega_k}} > 25.203983) = 2.127326e-65
$$

So, the conclusion for any $\hspace{0.1cm} \alpha > 2.127326e-65 \hspace{0.1cm}$ is **reject** $\hspace{0.1cm} H_0: M5\_py\hspace{0.1cm}$ in favor of $\hspace{0.1cm}H_1: M1\_py\hspace{0.1cm}$, so we can accept that $\hspace{0.1cm} M1\_py \hspace{0.1cm}$ is a **better** model than $\hspace{0.1cm} M5\_py \hspace{0.1cm}$

### F-test in `R` <a class="anchor" id="6"></a>

In [253]:
%%R

M1_R <- lm( price ~ size_in_m_2*quality  + no_of_bedrooms + no_of_bathrooms + latitude + longitude , data = data_R)

M2_R <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude , data = data_R)

M3_R <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + latitude + longitude , data = data_R)

M4_R <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms , data = data_R)

M5_R <- lm( price ~ size_in_m_2 , data = data_R)

In [254]:
%%R

anova(M2_R, M1_R)

Analysis of Variance Table

Model 1: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + 
    latitude + longitude
Model 2: price ~ size_in_m_2 * quality + no_of_bedrooms + no_of_bathrooms + 
    latitude + longitude
  Res.Df        RSS Df  Sum of Sq      F    Pr(>F)    
1   1896 4.8821e+15                                   
2   1893 4.6946e+15  3 1.8751e+14 25.204 5.483e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [255]:
%%R

anova(M3_R, M2_R)

Analysis of Variance Table

Model 1: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + latitude + 
    longitude
Model 2: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + 
    latitude + longitude
  Res.Df        RSS Df  Sum of Sq     F Pr(>F)  
1   1899 4.8987e+15                             
2   1896 4.8821e+15  3 1.6585e+13 2.147 0.0924 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [256]:
%%R

anova(M4_R, M3_R)

Analysis of Variance Table

Model 1: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms
Model 2: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + latitude + 
    longitude
  Res.Df        RSS Df  Sum of Sq      F    Pr(>F)    
1   1901 5.0803e+15                                   
2   1899 4.8987e+15  2 1.8163e+14 35.206 9.703e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [257]:
%%R

anova(M5_R, M1_R)

Analysis of Variance Table

Model 1: price ~ size_in_m_2
Model 2: price ~ size_in_m_2 * quality + no_of_bedrooms + no_of_bathrooms + 
    latitude + longitude
  Res.Df        RSS Df  Sum of Sq      F    Pr(>F)    
1   1903 5.5937e+15                                   
2   1893 4.6946e+15 10 8.9918e+14 36.258 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


### ANOVA test as a F-test <a class="anchor" id="7"></a>





Remember that the hypothesis of the ANOVA test are these:

\begin{gather*}
\hspace{-0.7 cm} H_0: \hspace{0.15cm} \beta_1=...=\beta_p=0 \\
H_1: \hspace{0.15cm} \exists \ j=1,...,p , \hspace{0.2cm} \beta_j \neq 0
\end{gather*}




Let us consider the following models:

  
-    $\Omega_k \ : \  \ y_i = \beta_0 + \beta_1\cdot x_{i1} +...+ \beta_{k-1}\cdot x_{i(k-1)} + \varepsilon_i$

-   $\omega_{q} \ : \ \ y_i = \beta_0 + \varepsilon_i \hspace{0.15cm}$ $(\hspace{0.05cm}
    Null \hspace{0.1cm} Model \hspace{0.05cm})$



Then, the ANOVA test is equivalent to the following:

\begin{gather*}
H_0: \hspace{0.15cm}  \ \hat{y}_i = \beta_0 + \beta_1\cdot x_{i1} +...+ \beta_{p}\cdot x_{ip} + \varepsilon_i  \ \hspace{0.2cm} ( \Omega_k ) \\
\hspace{-3.7cm} H_1: \hspace{0.15cm} \hat{y}_i = \beta_0 + \varepsilon_i  \ \hspace{0.2cm} ( \omega_{q} )
\end{gather*}




Where:

- $k=p+1$

- $q=1$

-   $RSS_{\Omega_{k=p+1}} = \sum_{i=1}^n ( y_i - \widehat{y}_{\hspace{0.01cm} \Omega_{k=p+1} \hspace{0.02cm},\hspace{0.02cm} i})^2 = \sum_{i=1}^n \left( y_i - ( \hat{\beta}_0 + \hat{\beta}_1\cdot x_{i1} +...+ \hat{\beta}_{p}\cdot x_{ip} ) \right)^2$

-   $RSS_{\omega_{q=1}} = \sum_{i=1}^n ( y_i - \widehat{y}_{\hspace{0.01cm} \omega_{q=1} \hspace{0.02cm},\hspace{0.02cm} i})^2 = \sum_{i=1}^n ( y_i - \hat{\beta}_0 )^2$

-   Note that in the null model $\hspace{0.1cm} \hat{\beta}_0=\overline{y} \hspace{0.1cm}$, therefore we have $\hspace{0.1cm} RSS_{\omega_{q=1}}=\sum_{i=1}^n ( y_i - \overline{y} )^2= TSS_{\omega_{q=1}}= TSS_{\Omega_{k=p+1}}=TSS$



More details about the ANOVA  terms (RSS, TSS, RegSS)  can be seen in the following article https://fabioscielzoortiz.github.io/Estadistica4all.github.io/Articulos/Linear%20Regression%20in%20Python%20and%20R.html



Using the above facts and the F-statistic we get the statistic test of the ANOVA test:

<br>

\begin{gather*}
F=\dfrac{(RSS_{\omega_{q=1}} - RSS_{\Omega_{k=p+1}})/(k-q)}{RSS_{\Omega_{k=p+1}}/(n-k)} = \dfrac{(TSS-RSS)/(p+1-1)}{RSS/(n-(p+1))} = \dfrac{(TSS-RSS)/p}{RSS/(n-p-1))} \sim F_{p+1-1 \hspace{0.03cm},\hspace{0.03cm} n-(p+1)} = F_{p \hspace{0.03cm},\hspace{0.03cm} n-p-1}
\end{gather*}





Where:

$TSS= RSS_{\omega_q}$

$RSS= RSS_{\Omega_k}$




And this is the test statistic of the ANOVA test, you can check that in the ANOVA test part of this article https://fabioscielzoortiz.github.io/Estadistica4all.github.io/Articulos/Linear%20Regression%20in%20Python%20and%20R.html

#### Anova test as an F-test in `Python`


In [258]:
full_model_py = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude', 
                        data =data_Python).fit()

null_model_py = smf.ols(formula = 'price ~ 1', data =data_Python).fit()

In [259]:
anova_lm(null_model_py , full_model_py)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,1904.0,1.615874e+16,0.0,NaN,NaN,NaN
1,1896.0,4.882082e+15,8.0,1.127666e+16,547.423881,0.0


#### Anova test as an F-test in `R`


In [260]:
%%R

full_model_R <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude , data = data_R)
null_model_R <- lm( price ~ 1 , data = data_R)

In [261]:
%%R

anova(null_model_R, full_model_R)

Analysis of Variance Table

Model 1: price ~ 1
Model 2: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + 
    latitude + longitude
  Res.Df        RSS Df  Sum of Sq      F    Pr(>F)    
1   1904 1.6159e+16                                   
2   1896 4.8821e+15  8 1.1277e+16 547.42 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


### Significance test as an F-test <a class="anchor" id="8"></a>






Remember that the hypothesis of the significance test of $\beta_j$ are these:

\begin{gather*}
H_0: \beta_j=0 \\
H_1: \beta_j \neq 0
\end{gather*}

Let us consider the following models:

-   $\omega_q  : \  \ y_i = \beta_0 + \beta_1\cdot x_{i1} +..+\beta_{j-1} \cdot x_{i,j-1}+\beta_{j+1} \cdot x_{i,j+1}+..+ \beta_{p}\cdot x_{ip} + \varepsilon_i$

-   $\Omega_k : \ \ y_i = \beta_0 + \beta_1\cdot x_{i1} +..+\beta_j \cdot x_{ij}+..+ \beta_{p}\cdot x_{ip} + \varepsilon_i$
    




Then, the significance test of $\beta_j$ is equivalent to the following:

\begin{gather*}
H_0: \hspace{0.2cm} y_i = \beta_0 + \beta_1\cdot x_{i1} +..+\beta_{j-1} \cdot x_{i,j-1}+\beta_{j+1} \cdot x_{i,j+1}+..+ \beta_{p}\cdot x_{ip} + \varepsilon_i \ \hspace{0.2cm} (\omega_q) \\
\hspace{-2.7cm} H_1: \hspace{0.2cm}  y_i = \beta_0 + \beta_1\cdot x_{i1} +...+\beta_j \cdot x_{ij}+...+ \beta_{p}\cdot x_{ip} + \varepsilon_i  \ \hspace{0.2cm} ( \Omega_k )
\end{gather*}




Where:

- $k=p+1$

- $q=k-1=p$



-   $RSS_{\Omega_{k=p+1}} = \sum_{i=1}^n ( y_i - \hat{y}_{\hspace{0.01cm} \Omega_k \hspace{0.02cm},\hspace{0.02cm} i})^2 = \sum_{i=1}^n\left( y_i -  ( \hat{\beta}_0 + \hat{\beta}_1\cdot x_{i1}  +... +  \hat{\beta}_{j} \cdot x_{i,j}+...+ \hat{\beta}_{p}\cdot x_{ip} )  \right)^2$

-   $RSS_{\omega_{q=p}} = \sum_{i=1}^n ( y_i - \hat{y}_{\hspace{0.01cm} \omega_k \hspace{0.02cm},\hspace{0.02cm} i})^2 = \sum_{i=1}^n \left( y_i - ( \hat{\beta}_0 + \hat{\beta}_1\cdot x_{i1}  +..+ \hat{\beta}_{j-1} \cdot x_{i,j-1} +  \hat{\beta}_{j+1} \cdot x_{i,j+1}+..+ \hat{\beta}_{p}\cdot x_{ip} ) \right)^2$




So, the statistic test is obtained applying the F-statistic formula:

\begin{gather*}
F=\dfrac{(RSS_{\omega_q} - RSS_{\Omega_k})/(k-q)}{RSS_{\Omega_k}/(n-k)} =\dfrac{(RSS_{\omega_{k-1}} - RSS_{\Omega_k})/(k-(k-1))}{RSS_{\Omega_k}/(n-k)} = \dfrac{(RSS_{\omega_{k-1}} - RSS_{\Omega_k})/1)}{RSS_{\Omega_k}/(n-p-1)}  \sim F_{k-q, n-k} = F_{1, n-p-1}
\end{gather*}



The results of the test using the F-test is approximately equal to the result obtained with the other alternative (t-test).

**Important**: $\hspace{0.1cm}$ this is the way to test the significance of categorical variables (compare the model without the categorical variable vs the model with it), and also to test the significance of two or more variables at the same time.

#### Significance test as an F-test in `Python`

$$
H_0: \beta_{quality} = 0 \\
H_1: \beta_{quality} \neq 0
$$

In [262]:
Model_with_quality_py =  smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude', data =data_Python).fit()
Model_without_quality_py =  smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + latitude + longitude', data =data_Python).fit()

In [263]:
anova_lm(Model_without_quality_py, Model_with_quality_py)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,1899.0,4.898667e+15,0.0,NaN,NaN,NaN
1,1896.0,4.882082e+15,3.0,1.658498e+13,2.146975,0.092395


$pvalor = 0.092395 \hspace{0.15cm}  \Rightarrow \hspace{0.15cm}$  if $\hspace{0.1cm} \alpha = 0.01 \hspace{0.15cm} or\hspace{0.15cm}  0.05 \hspace{0.15cm}$ we reject $\hspace{0.15cm} H_0: \beta_{quality} = 0 \hspace{0.15cm}$  , so we can accept $\hspace{0.15cm} H_1: \beta_{quality} \neq 0 \hspace{0.15cm}$ , so $quality$ is significant.

$$
\hspace{-1.4cm} H_0: \beta_{longitude}= \beta_{latitude} = 0 \\
H_1: \hspace{0.3cm} \beta_{longitude}\neq 0 \hspace{0.2cm} or \hspace{0.2cm} \beta_{latitude}\neq 0
$$

In [264]:
M1_py = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality', data=data_Python).fit()
M2_py = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude', data =data_Python).fit()

In [265]:
anova_lm(M1_py, M2_py)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,1898.0,5.066797e+15,0.0,NaN,NaN,NaN
1,1896.0,4.882082e+15,2.0,1.847150e+14,35.867848,5.131535e-16


$pvalor = 5.131535e-16 \hspace{0.15cm}  \Rightarrow \hspace{0.15cm}$   $\hspace{0.1cm} \forall \alpha  \hspace{0.15cm} $ we reject $\hspace{0.15cm} H_0: \beta_{longitude}= \beta_{latitude} = 0 \hspace{0.15cm}$  , so we can accept $\hspace{0.15cm} H_1: \beta_{longitude}\neq 0 \hspace{0.2cm} or \hspace{0.2cm} \beta_{latitude}\neq 0 \hspace{0.15cm}$ , so $\hspace{0.1cm} longitude \hspace{0.1cm}$ or $\hspace{0.1cm} latitude \hspace{0.1cm}$ is significant.

#### Significance test as an F-test in `R`

$$
H_0: \beta_{quality} = 0 \\
H_1: \beta_{quality} \neq 0
$$

In [266]:
%%R

Model_with_quality_R <- lm(price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude , data = data_R)
Model_without_quality_R <- lm(price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms  + latitude + longitude , data = data_R)

In [267]:
%%R

anova(Model_without_quality_R, Model_with_quality_R)

Analysis of Variance Table

Model 1: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + latitude + 
    longitude
Model 2: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + 
    latitude + longitude
  Res.Df        RSS Df  Sum of Sq     F Pr(>F)  
1   1899 4.8987e+15                             
2   1896 4.8821e+15  3 1.6585e+13 2.147 0.0924 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


$$
\hspace{-1.4cm} H_0: \beta_{longitude}= \beta_{latitude} = 0 \\
H_1: \hspace{0.3cm} \beta_{longitude}\neq 0 \hspace{0.2cm} or \hspace{0.2cm} \beta_{longitude}\neq 0
$$

In [268]:
%%R

M1_R <- lm('price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality', data=data_R)
M2_R <- lm('price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude', data =data_R)

In [269]:
%%R

anova(M1_R , M2_R)

Analysis of Variance Table

Model 1: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality
Model 2: price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + 
    latitude + longitude
  Res.Df        RSS Df  Sum of Sq      F    Pr(>F)    
1   1898 5.0668e+15                                   
2   1896 4.8821e+15  2 1.8471e+14 35.868 5.132e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


## Iterative Algorithms <a class="anchor" id="9"></a>

Now we are going to see several algorithms aimed at choosing a linear regression model over many or all possible ones, under certain criteria based in some metrics.

### Metrics <a class="anchor" id="10"></a>

These metrics are one of the most important concepts in modern statistics and machine learning. 

Some of them are:

- Test error by Cross validation 
- $\widehat{R}^2$  
- AIC
- BIC
- Cp



A detailed review of cross-validation methods will be done in a future article on my blog. The test error by cross validation will not be used as a criteria in our practical implementation of the following iterative algorithms, because in the iterative selection of linear regression models the most common criteria is to use AIC, BIC or $\widehat{R}^2$, so we will do a review of these last.

But in model selection in general, cross-validation plays a very prominent role.

####  $\widehat{R}^2$  <a class="anchor" id="11"></a>

This metric is explained with more details in the following article about linear regression :  

https://fabioscielzoortiz.github.io/Estadistica4all.github.io/Articulos/Linear%20Regression%20in%20Python%20and%20R.html

Here we will just show the formula that characterizes the adjusted $R^2$:

Given a linear regression model $\hspace{0.05cm} M \hspace{0.05cm}$  with $\hspace{0.05cm} p_M \hspace{0.051cm}$ predictors (independent variables) and $n$ observations:

\begin{gather*}
\widehat{R}^2 = 1 - \left( 1- R^2 \right) \cdot \dfrac{n-1}{n-p_M}
\end{gather*}

Where:

\begin{gather*}
R^2 = \dfrac{RegSS}{TSS}
\end{gather*}

This metric is usually used as a criteria to select linear regression models.

##### **$\widehat{R}^2$ criteria**: 

Given $h$ linear regression models $\hspace{0.1cm}M_1 , M_2, \dots, M_h$

If $\hspace{0.1cm}  \widehat{R}^2 (M_j) > \widehat{R}^2 (M_r) \hspace{0.2cm} , \forall r \in \lbrace 1,...,h\rbrace - \lbrace j \rbrace \hspace{0.2cm} \Rightarrow \hspace{0.2cm} $ $M_j$ is selected instead of $M_r$  $ \hspace{0.1cm} , \forall r \in \lbrace 1,...,h\rbrace - \lbrace j \rbrace$



That is, the model with the **highest** $\hspace{0.1cm} \widehat{R}^2 \hspace{0.1cm}$ is selected over the rest.


##### $\widehat{R}^2$ in `Python`

In [270]:
X = data_Python[['size_in_m_2', 'longitude', 'latitude', 'no_of_bedrooms', 'no_of_bathrooms', 'quality']]
y = data_Python['price']

In [271]:
model = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality', data=data_Python).fit()

In [272]:
model.rsquared_adj

0.6854449178693436

##### $\widehat{R}^2$ in `R`

In [273]:
%%R

model <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality , data = data_R)

In [274]:
%%R 

summary(model)$adj.r.squared

[1] 0.6854449


#### $AIC$ <a class="anchor" id="12"></a>

Given a linear regression model $\hspace{0.05cm} M \hspace{0.05cm}$  with $\hspace{0.05cm} p_M \hspace{0.051cm}$ predictors and $n$ observations:

$$ AIC(M) = -2 \cdot ln\left(\widehat{L}(M)\right) + 2 \cdot  \left(\hspace{0.1cm} p_M +1 \hspace{0.1cm}\right)  $$




Where:

$\widehat{L}(M)$ is the value of the likelihood function of the model $M$ evaluated at the MLE (Maximum Likelihood Estimators)



This metric is usually used as a criteria to select linear regression models.



##### **$AIC$ criteria**:

Given $h$ linear regression models $\hspace{0.1cm}M_1 , M_2, \dots, M_h$

If $\hspace{0.1cm}  AIC (M_j) < AIC(M_r) \hspace{0.2cm} , \forall r \in \lbrace 1,...,h\rbrace - \lbrace j \rbrace \hspace{0.2cm} \Rightarrow \hspace{0.2cm} $ $M_j$ is selected instead of $M_r$  $ \hspace{0.1cm} , \forall r \in \lbrace 1,...,h\rbrace - \lbrace j \rbrace$



That is, the model with the **less** $AIC$ is selected over the rest.


#### $BIC$  <a class="anchor" id="13"></a>

Given a linear regression model $\hspace{0.05cm} M \hspace{0.05cm}$  with $\hspace{0.05cm} p_M \hspace{0.051cm}$ predictors and $n$ observations:

$$ BIC(M) = -2 \cdot ln\left(\widehat{L}(M)\right) + ln(n) \cdot  \left(\hspace{0.1cm} p_M +1 \hspace{0.1cm}\right)  $$



Where:

$\widehat{L}(M)$ is the value of the likelihood function of the model $M$ evaluated at the MLE (Maximum Likelihood Estimators)

This metric is usually used as a criteria to select linear regression models.



$BIC$ **criteria**:


Given $h$ linear regression models $\hspace{0.1cm}M_1 , M_2, \dots, M_h$

If $\hspace{0.1cm}  BIC (M_j) < BIC(M_r) \hspace{0.2cm} , \forall r \in \lbrace 1,...,h\rbrace - \lbrace j \rbrace \hspace{0.2cm} \Rightarrow \hspace{0.2cm} $ $M_j$ is selected instead of $M_r$  $ \hspace{0.1cm} , \forall r \in \lbrace 1,...,h\rbrace - \lbrace j \rbrace$

That is, the model with the **less** $BIC$ is selected over the rest.

### Maximum Likelihood Estimation in the Linear Regression Model <a class="anchor" id="14"></a>


Given a linear regression model $\hspace{0.1cm} M \hspace{0.1cm}$  with $\hspace{0.1cm} p_M \hspace{0.1cm}$ predictors and $n$ observations:

$$ y_i \sim N(\hspace{0.1cm} x_i^t  \cdot \beta \hspace{0.1cm} , \hspace{0.1cm} \sigma^2 \hspace{0.1cm} )$$

$$ y_i \sim f(y_i) = \dfrac{1}{\sqrt{2\pi \sigma^2}} \cdot exp\lbrace \hspace{0.1cm} - \dfrac{1}{2\sigma^2} \cdot (y_i - x^t_i\cdot \beta)^2 \hspace{0.1cm} \rbrace $$


The likelihood function of the model $M$ is:

$$ L(  M )=L(\beta, \sigma \hspace{0.1cm}|\hspace{0.1cm} x_i, y_i)= \prod_{i=1}^{n} f(y_i) = (2\pi \sigma^2)^{-n/2} \cdot exp\lbrace \hspace{0.1cm} - \dfrac{1}{2\sigma^2}\cdot \sum_{i=1}^{n} (y_i - x^t_i\cdot \beta)^2 \hspace{0.1cm} \rbrace$$

Taking natural logarithm we have:

$$ln\left(\hspace{0.1cm}L(M)\hspace{0.1cm}\right)=ln(\hspace{0.1cm} L(\beta, \sigma \hspace{0.1cm}|\hspace{0.1cm} x_i, y_i)\hspace{0.1cm}) = - \dfrac{n}{2} \left(ln(2\pi) + ln(\sigma^2) \right) - \dfrac{1}{2\sigma^2} \sum_{i=1}^{n} \left( y_i - x^t_i\cdot \beta \right) ^2   $$



The maximum likelihood estimators  of the parameters $\hspace{0.1cm} \beta$ , $\sigma \hspace{0.1cm}$ of the linear regression model $M$ are calculated as the solution of the following optimization problem:

$$
\underset{\beta \hspace{0.05cm},\hspace{0.05cm} \sigma}{Max} \hspace{0.2cm} ln(\hspace{0.1cm}L(M)\hspace{0.1cm})
$$




Solutions:

$$
\hat{\beta}_{MLE}=(X^t \cdot X)^{-1} \cdot X^t \cdot Y = \hat{\beta}_{OLS}
$$
$$
\hat{\sigma}^2_{MLE} = \dfrac{RSS(M)}{n}
$$




Note that:

$$
arg \hspace{0.1cm} \underset{\beta \hspace{0.05cm},\hspace{0.05cm} \sigma}{Max} \hspace{0.2cm} L(M) \hspace{0.1cm}=\hspace{0.1cm} arg \hspace{0.1cm} \underset{\beta \hspace{0.05cm},\hspace{0.05cm} \sigma}{Max} \hspace{0.2cm} ln(\hspace{0.1cm}L(M)\hspace{0.1cm}) 
$$



So, the function $\hspace{0.1cm} ln \left( \hspace{0.1cm}L(M)\hspace{0.1cm} \right) \hspace{0.1cm}$ evaluated in $\hspace{0.1cm} \beta=\hat{\beta}_{MLE} \hspace{0.1cm}, \hspace{0.1cm}\sigma^2 = \hat{\sigma}^2_{MLE} \hspace{0.1cm}$ is :



$$ ln \left( \hspace{0.1cm}\widehat{L}(M)\hspace{0.1cm} \right) =  - \dfrac{n}{2} \left( ln(2\pi) + ln\left(\dfrac{RSS(M)}{n}\right) - ln(n) + 1 \right) $$



 #### $AIC$ and $BIC$ in the Linear Regression Model <a class="anchor" id="14.1"></a> 


Then, in the linear regression model:



$$ AIC(M) = n \cdot \left( \hspace{0.1cm}  ln(2\pi) + ln(RSS(M)) - ln(n) \hspace{0.1cm} \right) + n + 2\cdot (\hspace{0.1cm}p_M + 1\hspace{0.1cm}) $$

$$ BIC(M) =  n \cdot \left(  \hspace{0.1cm} ln(2\pi) + ln(RSS(M)) - ln(n) \hspace{0.1cm} \right) + n + ln(n)\cdot(\hspace{0.1cm} p_M + 1\hspace{0.1cm}) $$



Where:

$$
RSS(M) =  \sum_{i=1}^n \hat{\varepsilon}_i^2 =  \sum_{i=1}^n ( y_i - \hat{y}_i)^2
$$

#### $AIC$ in `Python` <a class="anchor" id="14"></a>

Using `smf.ols` :

In [275]:
model = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms+ longitude + latitude + quality', data=data_Python).fit()

In [276]:
model.aic

59854.027050178316

Using the $AIC$ formula: 

In [277]:
def AIC_lm(data, model):

    import math

    n = data.shape[0]

    p = model.df_model

    pi = math.pi

    RSS = anova_lm(model).sum_sq.Residual

    # Other way to compute RSS:
    
    # y = data['response_variable']
    # RSS = ( ( y - model.predict(X) )**2 ).sum()

    AIC = n*(math.log(2*pi) + math.log(RSS) -  math.log(n)) + n + 2*(p+1)

    return AIC

In [278]:
model = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms+ longitude + latitude + quality', data=data_Python).fit()

In [279]:
AIC_lm(data_Python, model)

59854.02705017832

Note:

`math.log(x , base)` , by default $\hspace{0.1cm} base=e$

$ln(x) =$ `math.log(x)` $=$ `math.log(x , math.e)`   

#### $AIC$ in `R` <a class="anchor" id="14"></a>

In [280]:
%%R

model <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality , data = data_R)

In [281]:
%%R

AIC(model)

[1] 59922.77


##### $BIC$ in `Python` <a class="anchor" id="15"></a>

Using `smf.ols` :

In [282]:
model = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality', data=data_Python).fit()

In [283]:
model.bic

59959.63885835402

Using the $BIC$ formula: 

In [284]:
def BIC_lm(data, model):

    import math

    n = data.shape[0]

    p = model.df_model

    pi = math.pi

    RSS = anova_lm(model).sum_sq.Residual

    # Other way to compute RSS:
    
    # y = data['response_variable']
    # RSS = ( ( y - model.predict(X) )**2 ).sum()

    BIC = n*(math.log(2*pi) + math.log(RSS) -  math.log(n)) + n + math.log(n)*(p+1)

    return BIC

In [285]:
model = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality', data=data_Python).fit()

In [286]:
BIC_lm(data_Python, model)

59959.638858354025

##### $BIC$ in `R` <a class="anchor" id="15"></a>

In [287]:
%%R

model <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality , data = data_R)

In [288]:
%%R

BIC(model)

[1] 59967.19


#### $C_p$ <a class="anchor" id="18"></a>

Given a full linear regression model  with $\hspace{0.1cm} p\hspace{0.1cm}$ predictors $\hspace{0.1cm} M_{Full}:\hspace{0.1cm} y_i = \beta_0 + \sum_{j=1}^{p} \hspace{0.02cm} \beta_j \cdot x_{ij}$

Given a linear regression model $\hspace{0.1cm} M \subseteq M_{Full}\hspace{0.1cm}$  with $\hspace{0.1cm} p_M \leq p\hspace{0.1cm}$ predictors and $n$ observations:



$$ C_p(M) = \dfrac{RSS(M)}{\hat{\sigma}_{M_{Full}}^2} - n + 2\cdot \left(\hspace{0.1cm} p_M +1 \hspace{0.1cm}\right) $$



Where:

$ \hat{\sigma}_{M_{Full}}^2 =  \dfrac{RSS(M_{Full})}{n-p-1}  \hspace{0.2cm} $ is the residual variance of the full model.

$
RSS(M_{Full}) =    \sum_{i=1}^{n} (y_i - \widehat{y}_{M_{Full}\hspace{0.02cm},\hspace{0.02cm} i})^2
$

#### $C_p$ in `Python` <a class="anchor" id="14"></a>

In [289]:
def Cp_lm(data, full_model, model):

    # full_model and model have to be a  smf.ols, because `anova_lm` doesn´t work with sm.OLS

    n = data.shape[0]

    p_full_model =  full_model.df_model

    p_model =  model.df_model

    RSS_full_model = anova_lm(full_model).sum_sq.Residual

    RSS_model = anova_lm(model).sum_sq.Residual

    residual_variance_full_model = RSS_full_model/(n-p_full_model-1)

    # Other way to compute RSS:
    
    # y = data['response_variable']
    # RSS = ( ( y - model.predict(X) )**2 ).sum()

    Cp = RSS_model/residual_variance_full_model - n + 2*(p_model + 1)

    return Cp

In [290]:
full_model = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude', data =data_Python).fit()

model = smf.ols(formula = 'price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms', data =data_Python).fit()

In [291]:
Cp_lm(data_Python, full_model, model)

75.98029458609994

#### $C_p$ in `R` <a class="anchor" id="14"></a>

In [292]:
%%R

# install.packages("olsrr")

NULL


In [293]:
%%R

full_model <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms + quality + latitude + longitude  , data = data_R)

model <- lm( price ~ size_in_m_2 + no_of_bedrooms + no_of_bathrooms  , data = data_R)

In [294]:
%%R

library(olsrr)

ols_mallows_cp(model, full_model)

[1] 75.98029


### Best Subset Selection <a class="anchor" id="20"></a>

Best subset selection  consist in the following algorithm :

We have $p$ predictors: $\hspace{0.1cm} X_1,...,X_p$

- We train the null linear model  
   
- We train all the possible linear models with  $1$ predictor 

- We train all the possible linear models with $2$ predictors
  
   $\dots$ 

- We train all the possible linear models with $\hspace{0.05cm}p-1\hspace{0.05cm}$ predictors

- We train the full linear model 


We select one of those models under some criteria, for example the one with **less**  $Train\_error$ , $AIC$, $BIC$ or $Cp$, or **greater**  $\widehat{R}^2$. That model will be consider "the best model".




Scheme of the algorithm:

- Train $\hspace{0.08cm}M_0 = \lbrace \text{null model} \rbrace$

- Train $\hspace{0.08cm}M_1=\lbrace \text{models with 1 predictor} \rbrace  $

- Train $\hspace{0.08cm}M_2=\lbrace \text{models with 2 predictors} \rbrace  $

$\hspace{0.8cm} \dots$

- Train $\hspace{0.08cm}M_{p-1}=\lbrace \text{models with p-1 predictors} \rbrace  $

- Train $\hspace{0.08cm}M_{p}=\lbrace \text{full model} \rbrace  $



- $\lbrace M_0, M_1 , ..., M_p \rbrace \underset{ Train\_error, AIC, BIC, C_p, \widehat{R}^2 }{\Rightarrow} \hspace{0.1cm} M\hspace{0.05cm}^* \hspace{0.1cm}(Best \hspace{0.1cm} Model) $

**Problems:**

- **Large computational requirements:** compute $2^p$ models is required, which is impossible with more than $\hspace{0.05cm} p=40 \hspace{0.05cm}$ predictors, because $\hspace{0.05cm} 2^{40}=1099511627776$.
  
  The growth of the computational requirements is exponential, for example, with $\hspace{0.05cm} p=10$ predictors we need to calculate $\hspace{0.05cm} 2^{10}=1024 \hspace{0.05cm}$ models, but with $\hspace{0.05cm} p=15\hspace{0.05cm} $ the models to be calculated are too many, $\hspace{0.05cm} 2^{15}=32768$

### Best Subset Selection in `Python` <a class="anchor" id="21"></a>

We are going to define a function to process categorical variables:

In [295]:
def varcharProcessing(X, varchar_process = "dummy_dropfirst"):
    
    dtypes = X.dtypes

    if varchar_process == "drop":   
        X = X.drop(columns = dtypes[dtypes == np.object].index.tolist())

    elif varchar_process == "dummy":
        X = pd.get_dummies(X,drop_first=False)

    elif varchar_process == "dummy_dropfirst":
        X = pd.get_dummies(X,drop_first=True)

    else: 
        X = pd.get_dummies(X,drop_first=True)
    
    X["intercept"] = 1
    cols = X.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    X = X[cols]
    
    return X

Let's see how the function `varcharProcessing` works:

In [296]:
X = data_Python[['size_in_m_2', 'longitude', 'latitude', 'no_of_bedrooms', 'no_of_bathrooms', 'quality']]
y = data_Python['price']

In [297]:
X

,size_in_m_2,longitude,latitude,no_of_bedrooms,no_of_bathrooms,quality
0,100.242337,55.138932,25.113208,1,2,1
1,146.972546,55.151201,25.106809,2,2,1
2,181.253753,55.137728,25.063302,3,5,1
3,187.664060,55.341761,25.227295,2,3,0
4,47.101821,55.139764,25.114275,0,1,1
...,...,...,...,...,...,...
1900,100.985561,55.310712,25.176892,2,2,3
1901,70.606280,55.276684,25.166145,1,2,1
1902,179.302790,55.345056,25.206500,3,5,1
1903,68.748220,55.229844,25.073858,1,2,1


The function `varcharProcessing` has two inputs, first a data-set, and second three options (**drop**, **dummy**, **dummy_dropfirst**).



- If the second input is **drop**, the function drops all the variables object type.



- If the second input is **dummy**, the function drops the original categorical variables, and generate the dummy variables associated to their categories 

For example: $quality$ would be dropped, and because of $quality$ has 4 categories (0,1,2,3) the dummies variables $quality\_0$ , $quality\_1$, $quality\_2$, $quality\_3$ would be added to the date-set.



- If the second input is **dummy_dropfirst**, 
the function drops the original categorical variable, and generate the dummy variables associated to their categories , but dropping the dummy associated with the first category.

For example: $quality$ would be dropped, and because of $quality$ has 4 categories (0,1,2,3) and  first dummy must be dropped,  the dummies variables $quality\_1$, $quality\_2$, $quality\_3$ would be added to the date-set.

- In all the cases the function add the intercept column to the data-set

#### Note:

#### Dummy variables associated to a categorical variable:

$D(X_j , i)$ is the dummy variable associated to the category $i$ of the categorical variable $X_j$ $\Leftrightarrow$

$$
\Leftrightarrow D(X_j , i) = \left\lbrace\begin{array}{l} 1 \hspace{0.3cm},\text{ if $\hspace{0.1cm} X_j = i$ } \\ 0 \hspace{0.3cm},\text{   if $\hspace{0.1cm} X_j \neq i$  }  \end{array}\right.
$$

Let´s see the above :

In [298]:
varcharProcessing(X, varchar_process = "dummy_dropfirst")

,intercept,size_in_m_2,longitude,latitude,no_of_bedrooms,no_of_bathrooms,quality_1,quality_2,quality_3
0,1,100.242337,55.138932,25.113208,1,2,1,0,0
1,1,146.972546,55.151201,25.106809,2,2,1,0,0
2,1,181.253753,55.137728,25.063302,3,5,1,0,0
3,1,187.664060,55.341761,25.227295,2,3,0,0,0
4,1,47.101821,55.139764,25.114275,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...
1900,1,100.985561,55.310712,25.176892,2,2,0,0,1
1901,1,70.606280,55.276684,25.166145,1,2,1,0,0
1902,1,179.302790,55.345056,25.206500,3,5,1,0,0
1903,1,68.748220,55.229844,25.073858,1,2,1,0,0


In [299]:
varcharProcessing(X, varchar_process = "dummy")

,intercept,size_in_m_2,longitude,latitude,no_of_bedrooms,no_of_bathrooms,quality_0,quality_1,quality_2,quality_3
0,1,100.242337,55.138932,25.113208,1,2,0,1,0,0
1,1,146.972546,55.151201,25.106809,2,2,0,1,0,0
2,1,181.253753,55.137728,25.063302,3,5,0,1,0,0
3,1,187.664060,55.341761,25.227295,2,3,1,0,0,0
4,1,47.101821,55.139764,25.114275,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1900,1,100.985561,55.310712,25.176892,2,2,0,0,0,1
1901,1,70.606280,55.276684,25.166145,1,2,0,1,0,0
1902,1,179.302790,55.345056,25.206500,3,5,0,1,0,0
1903,1,68.748220,55.229844,25.073858,1,2,0,1,0,0


In [300]:
varcharProcessing(X, varchar_process = "drop")

,intercept,size_in_m_2,longitude,latitude,no_of_bedrooms,no_of_bathrooms,quality
0,1,100.242337,55.138932,25.113208,1,2,1
1,1,146.972546,55.151201,25.106809,2,2,1
2,1,181.253753,55.137728,25.063302,3,5,1
3,1,187.664060,55.341761,25.227295,2,3,0
4,1,47.101821,55.139764,25.114275,0,1,1
...,...,...,...,...,...,...,...
1900,1,100.985561,55.310712,25.176892,2,2,3
1901,1,70.606280,55.276684,25.166145,1,2,1
1902,1,179.302790,55.345056,25.206500,3,5,1
1903,1,68.748220,55.229844,25.073858,1,2,1


In last one, if quality would be object type, it would be drop. But in this case it has category type, so it hasn't been dropped.

In [301]:
X.dtypes


size_in_m_2         float64
longitude           float64
latitude            float64
no_of_bedrooms        int64
no_of_bathrooms       int64
quality            category
dtype: object

What happened if we have a categorical variable with a not standard range ?

No problem, `varcharProcessing` keeps working well, following the same philosophy.

Let's see that:

In [302]:
X['quality'] = X['quality'].astype(float) 

X['quality'] = X['quality'] + 2

X['quality'] = X['quality'].astype('category') 

In [303]:
varcharProcessing(X , varchar_process = "dummy")

,intercept,size_in_m_2,longitude,latitude,no_of_bedrooms,no_of_bathrooms,quality_2.0,quality_3.0,quality_4.0,quality_5.0
0,1,100.242337,55.138932,25.113208,1,2,0,1,0,0
1,1,146.972546,55.151201,25.106809,2,2,0,1,0,0
2,1,181.253753,55.137728,25.063302,3,5,0,1,0,0
3,1,187.664060,55.341761,25.227295,2,3,1,0,0,0
4,1,47.101821,55.139764,25.114275,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1900,1,100.985561,55.310712,25.176892,2,2,0,0,0,1
1901,1,70.606280,55.276684,25.166145,1,2,0,1,0,0
1902,1,179.302790,55.345056,25.206500,3,5,0,1,0,0
1903,1,68.748220,55.229844,25.073858,1,2,0,1,0,0


Now we can undo the previous changes:

In [304]:
X['quality'] = X['quality'].astype(float) 

X['quality'] = X['quality'] - 2

X['quality'] = X['quality'].astype('category') 

In the following algorithms we are going to use `statsmodels.api` instead of `statsmodels.formula.api` to implement the linear regression. So we have to take into account some particularities of  `statsmodels.api`

One of the most important differences is that `statsmodels.api` doesn´t fit the intercept, while `statsmodels.formula.api` does.

Another important difference is that `statsmodels.api` doesn't deal well with categorical predictors, while `statsmodels.formula.api` does.

In [305]:
import statsmodels.api as sm

For example, the following model doesn't fit the intercept neither deal well the categorical predictor $quality$

In [306]:
X = data_Python[['size_in_m_2', 'longitude', 'latitude', 'no_of_bedrooms', 'no_of_bathrooms', 'quality']]
y = data_Python['price']

In [307]:
model = sm.OLS( y , X ).fit()

print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.800
Model:                            OLS   Adj. R-squared (uncentered):              0.799
Method:                 Least Squares   F-statistic:                              1264.
Date:               ma., 06 sep. 2022   Prob (F-statistic):                        0.00
Time:                        11:35:12   Log-Likelihood:                         -29920.
No. Observations:                1905   AIC:                                  5.985e+04
Df Residuals:                    1899   BIC:                                  5.989e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

To fit the intercept and properly treat  categorical predictors with `statsmodels.api` we need to add the intercept and the dummy (binary) variables associated with the categorical predictors to $X$ as new predictors, and this can be done using `varcharProcessing `


You can get more information about how categorical predictors must be deal in linear regression models in the following article: 

https://fabioscielzoortiz.github.io/Estadistica4all.github.io/Articulos/Linear%20Regression%20in%20Python%20and%20R.html


In [308]:
X_new = varcharProcessing(X, varchar_process = "dummy_dropfirst")

In [309]:
X_new

,intercept,size_in_m_2,longitude,latitude,no_of_bedrooms,no_of_bathrooms,quality_1,quality_2,quality_3
0,1,100.242337,55.138932,25.113208,1,2,1,0,0
1,1,146.972546,55.151201,25.106809,2,2,1,0,0
2,1,181.253753,55.137728,25.063302,3,5,1,0,0
3,1,187.664060,55.341761,25.227295,2,3,0,0,0
4,1,47.101821,55.139764,25.114275,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...
1900,1,100.985561,55.310712,25.176892,2,2,0,0,1
1901,1,70.606280,55.276684,25.166145,1,2,1,0,0
1902,1,179.302790,55.345056,25.206500,3,5,1,0,0
1903,1,68.748220,55.229844,25.073858,1,2,1,0,0


In [310]:
model = sm.OLS(y , X_new ).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     547.4
Date:               ma., 06 sep. 2022   Prob (F-statistic):               0.00
Time:                        11:35:13   Log-Likelihood:                -29918.
No. Observations:                1905   AIC:                         5.985e+04
Df Residuals:                    1896   BIC:                         5.990e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
intercept       -6.207e+07   2.99e+07     

To build the best subset selection algorithm, we have to explore all possible combinations of predictors, for $1,2,3,...,p \hspace{0.1cm}$ predictors .

This task is very complex, but luckily in Python we have the `itertools` package which has iteration tools to help us.

In [311]:
# pip install tqdm

In [312]:
import itertools

from tqdm import tnrange

Let's see how `itertools.combinations` works:

In [313]:
# pip install ipywidgets

In [314]:
for k in tnrange( 1, len(X.columns) + 1, desc = 'Loop...'):

    for combo in itertools.combinations(X.columns , k): 

        print(combo)

# itertools.combinations(X.columns , k) generate all the possible combinations with k columns of X (predictors)

Loop...:   0%|          | 0/6 [00:00<?, ?it/s]

('size_in_m_2',)
('longitude',)
('latitude',)
('no_of_bedrooms',)
('no_of_bathrooms',)
('quality',)
('size_in_m_2', 'longitude')
('size_in_m_2', 'latitude')
('size_in_m_2', 'no_of_bedrooms')
('size_in_m_2', 'no_of_bathrooms')
('size_in_m_2', 'quality')
('longitude', 'latitude')
('longitude', 'no_of_bedrooms')
('longitude', 'no_of_bathrooms')
('longitude', 'quality')
('latitude', 'no_of_bedrooms')
('latitude', 'no_of_bathrooms')
('latitude', 'quality')
('no_of_bedrooms', 'no_of_bathrooms')
('no_of_bedrooms', 'quality')
('no_of_bathrooms', 'quality')
('size_in_m_2', 'longitude', 'latitude')
('size_in_m_2', 'longitude', 'no_of_bedrooms')
('size_in_m_2', 'longitude', 'no_of_bathrooms')
('size_in_m_2', 'longitude', 'quality')
('size_in_m_2', 'latitude', 'no_of_bedrooms')
('size_in_m_2', 'latitude', 'no_of_bathrooms')
('size_in_m_2', 'latitude', 'quality')
('size_in_m_2', 'no_of_bedrooms', 'no_of_bathrooms')
('size_in_m_2', 'no_of_bedrooms', 'quality')
('size_in_m_2', 'no_of_bathrooms', 'qua

`itertools.combinations(X.columns , k)` generate all the possible combinations with $k$ columns of $X$ (`X.columns`), and $k$ goes from $1$ to the number of columns of $X$

We are going to develop the Best Subset Selection algorithm in `Python` :

In [318]:
def BestSubset (X , y,  varchar_process="dummy_dropfirst", criteria="AIC"):

    X =  varcharProcessing(X, varchar_process = varchar_process)
 

    AIC, BIC , ADJ_R2 , TRAIN_ERROR_MSE = [],[],[],[]
    
    predictor_list = []
    num_predictor = []

    cols = X.columns[1:len(X.columns)] # We are not selecting the intercept


######################################################################################################################################################

   # Null Model:

    model = sm.OLS(y, X[ ['intercept'] ] ).fit()

    TRAIN_ERROR_MSE.append( ( (y - model.predict( X['intercept'] ))**2 ).mean()  )
    AIC.append(model.aic)
    BIC.append(model.bic)
    ADJ_R2.append(model.rsquared_adj)

    predictor_list.append(['intercept'] )
    num_predictor.append(len(['intercept'] )) 

######################################################################################################################################################

  # All possible models:

    #Looping over  1 to p predictors in X
    for k in tnrange(1, len(cols) + 1, desc = 'Loop...'):

        #Looping over all possible combinations: from p choose k
        for combo in itertools.combinations(cols , k):

            model = sm.OLS(y, X[ ['intercept'] + list(combo) ] ).fit()

            TRAIN_ERROR_MSE.append( ( (y - model.predict(X[  ['intercept'] + list(combo) ]))**2 ).mean()  )
            AIC.append(model.aic)
            BIC.append(model.bic)
            ADJ_R2.append(model.rsquared_adj)

            predictor_list.append(['intercept'] + list(combo))
            num_predictor.append(len(['intercept'] + list(combo)))   

######################################################################################################################################################

  #Store in DataFrame:

    Models = pd.DataFrame({"num_predictors": num_predictor, "model":predictor_list, "train_errors_MSE":TRAIN_ERROR_MSE, "AIC": AIC, "BIC":BIC,  "ADJ_R2":ADJ_R2})

    if (criteria == "AIC"):

        Models = Models.sort_values(by=["AIC"]).reset_index(drop=False)

        Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 
        
    elif (criteria == "BIC"):

        Models = Models.sort_values(by=["BIC"]).reset_index(drop=False)

        Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 

    elif (criteria == "ADJ_R2"):

        Models = Models.sort_values(by=["ADJ_R2"], ascending=False).reset_index(drop=False)
                  
        Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 

    elif (criteria == "TRAIN_ERROR_MSE"):

        Models = Models.sort_values(by=["TRAIN_ERROR_MSE"]).reset_index(drop=False)
                  
        Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit()
    
    elif ( criteria == "none"):

        Final_Model = "not compute, use some criteria (AIC, BIC, Adj.R2 or train_error_MSE)"
    

    return Models , Final_Model

Let's check how the function `BestSubset` works :

In [319]:
Models , Final_Model = BestSubset (X , y, varchar_process="dummy_dropfirst", criteria="AIC")

Loop...:   0%|          | 0/8 [00:00<?, ?it/s]

In [320]:
Models

,index,num_predictors,model,train_errors_MSE,AIC,BIC,ADJ_R2
0,165,6,"[intercept, size_in_m_2, longitude, latitude, ...",2.568593e+12,59852.348316,59885.661739,6.963840e-01
1,222,7,"[intercept, size_in_m_2, longitude, latitude, ...",2.566132e+12,59852.522224,59891.387885,6.965151e-01
2,250,8,"[intercept, size_in_m_2, longitude, latitude, ...",2.563718e+12,59852.729811,59897.147709,6.966407e-01
3,93,5,"[intercept, size_in_m_2, longitude, latitude, ...",2.572226e+12,59853.041370,59880.802556,6.961145e-01
4,224,7,"[intercept, size_in_m_2, longitude, latitude, ...",2.567521e+12,59853.553086,59892.418747,6.963508e-01
...,...,...,...,...,...,...,...
251,20,3,"[intercept, longitude, quality_2]",8.452975e+12,62115.508593,62132.165305,2.406893e-03
252,6,2,"[intercept, quality_1]",8.472054e+12,62117.803515,62128.907990,6.806342e-04
253,0,1,[intercept],8.482279e+12,62118.101353,62123.653590,-2.220446e-16
254,19,3,"[intercept, longitude, quality_1]",8.470115e+12,62119.367625,62136.024336,3.839818e-04


In [321]:
from IPython.display import HTML 

HTML(Models[0:7].to_html(classes='table table-striped'))

,index,num_predictors,model,train_errors_MSE,AIC,BIC,ADJ_R2
0,165,6,"[intercept, size_in_m_2, longitude, latitude, no_of_bedrooms, quality_2]",2.568593e+12,59852.348316,59885.661739,0.696384
1,222,7,"[intercept, size_in_m_2, longitude, latitude, no_of_bedrooms, quality_1, quality_2]",2.566132e+12,59852.522224,59891.387885,0.696515
2,250,8,"[intercept, size_in_m_2, longitude, latitude, no_of_bedrooms, quality_1, quality_2, quality_3]",2.563718e+12,59852.729811,59897.147709,0.696641
3,93,5,"[intercept, size_in_m_2, longitude, latitude, no_of_bedrooms]",2.572226e+12,59853.041370,59880.802556,0.696114
4,224,7,"[intercept, size_in_m_2, longitude, latitude, no_of_bedrooms, quality_2, quality_3]",2.567521e+12,59853.553086,59892.418747,0.696351
5,220,7,"[intercept, size_in_m_2, longitude, latitude, no_of_bedrooms, no_of_bathrooms, quality_2]",2.567841e+12,59853.790633,59892.656294,0.696313
6,247,8,"[intercept, size_in_m_2, longitude, latitude, no_of_bedrooms, no_of_bathrooms, quality_1, quality_2]",2.565462e+12,59854.024862,59898.442760,0.696434


So, the linear regression model selected with ***best subset*** algorithm and the $AIC$ criteria is saved in  `Final_Model`, and it is:

$price = intercept + size\_in\_m\_2 + longitude + latitude + no\_of\_bedrooms + quality\_2$

In [322]:
print(Final_Model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     874.4
Date:               ma., 06 sep. 2022   Prob (F-statistic):               0.00
Time:                        11:35:25   Log-Likelihood:                -29920.
No. Observations:                1905   AIC:                         5.985e+04
Df Residuals:                    1899   BIC:                         5.989e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
intercept      -6.098e+07   2.99e+07     -2.

Let us now try another criteria, for example $\widehat{R}^2$

In [323]:
Models , Final_Model = BestSubset (X , y, varchar_process="dummy_dropfirst", criteria="ADJ_R2")

Loop...:   0%|          | 0/8 [00:00<?, ?it/s]

In [324]:
Models

,index,num_predictors,model,train_errors_MSE,AIC,BIC,ADJ_R2
0,250,8,"[intercept, size_in_m_2, longitude, latitude, ...",2.563718e+12,59852.729811,59897.147709,6.966407e-01
1,255,9,"[intercept, size_in_m_2, longitude, latitude, ...",2.562773e+12,59854.027050,59903.997186,6.965926e-01
2,222,7,"[intercept, size_in_m_2, longitude, latitude, ...",2.566132e+12,59852.522224,59891.387885,6.965151e-01
3,247,8,"[intercept, size_in_m_2, longitude, latitude, ...",2.565462e+12,59854.024862,59898.442760,6.964344e-01
4,165,6,"[intercept, size_in_m_2, longitude, latitude, ...",2.568593e+12,59852.348316,59885.661739,6.963840e-01
...,...,...,...,...,...,...,...
251,20,3,"[intercept, longitude, quality_2]",8.452975e+12,62115.508593,62132.165305,2.406893e-03
252,6,2,"[intercept, quality_1]",8.472054e+12,62117.803515,62128.907990,6.806342e-04
253,19,3,"[intercept, longitude, quality_1]",8.470115e+12,62119.367625,62136.024336,3.839818e-04
254,0,1,[intercept],8.482279e+12,62118.101353,62123.653590,-2.220446e-16


### Best Subset Selection in `R` <a class="anchor" id="22"></a>

In [ ]:
%%R

# install.packages("olsrr")

In [344]:
%%R

model <- lm(price  ~ size_in_m_2  + longitude  + latitude + no_of_bedrooms  + no_of_bathrooms + quality , data = data_R)

ols_step_best_subset(model)

                              Best Subsets Regression                               
------------------------------------------------------------------------------------
Model Index    Predictors
------------------------------------------------------------------------------------
     1         size_in_m_2                                                           
     2         size_in_m_2 no_of_bedrooms                                            
     3         size_in_m_2 latitude no_of_bedrooms                                   
     4         size_in_m_2 latitude no_of_bedrooms quality                           
     5         size_in_m_2 longitude latitude no_of_bedrooms quality                 
     6         size_in_m_2 longitude latitude no_of_bedrooms no_of_bathrooms quality 
------------------------------------------------------------------------------------

                                                                 Subsets Regression Summary                          

### Forward Selection <a class="anchor" id="23"></a>

Forward selection  consist in the following algorithm :

We have $p$ predictors: $X_1,...,X_p$ and a response variable $Y$

- We train the null linear model $(M_0)$ 
  
- We train all the  linear models that are the result of *add one predictor* to the model $M_0$ , and we select one $(M_1)$ under some criteria, for example the one with **less** $train \hspace{0.1cm} error$.


- We train all the linear models that are the result of *add one predictor* to the model $M_1$ , and we select one $(M_2)$ under the same criteria.
  
   $\dots$ 

- We train all the linear models that are the result of *add one predictor* to the model $M_{p-2}$ , and we select one $(M_{p-1})$ under the same criteria.

- We train the full linear model $(M_p)$


We select one of the models $\hspace{0.1cm} \lbrace M_0, M_1,...,M_{p-1},M_p \rbrace \hspace{0.1cm}$ under some criteria, for example the one with **less** $\hspace{0.1cm} train\hspace{0.1cm} error$, $AIC$, $BIC$ or $Cp \hspace{0.1cm}$, or **greater**  $\hspace{0.1cm} \widehat{R}^2$ 


Scheme of the algorithm:

- $M_0$

- $\lbrace  M_0 \hspace{0.1cm} \text{+ 1 predictor} \rbrace \underset{ \text{train  error} }{\Rightarrow}M_1$

- $\lbrace  M_1 \hspace{0.1cm} \text{+ 1 predictor} \rbrace \underset{ \text{train  error} }{\Rightarrow}M_2$

$\hspace{0.8cm} \dots$

- $\lbrace  M_{p-2} \hspace{0.1cm} \text{+ 1 predictor} \rbrace \underset{ \text{train  error} }{\Rightarrow}M_{p-1}$

- $M_p$

- $\lbrace M_0, M_1 , ..., M_{p-1}, M_p \rbrace \underset{ \text{train  error}, AIC, BIC, C_p, \widehat{R}^2 }{\Rightarrow} \hspace{0.1cm} M\hspace{0.05cm}^* \hspace{0.1cm} (Best \hspace{0.1cm} Model) $

Where:

$\lbrace  M_j \hspace{0.1cm} \text{+ 1 predictor} \rbrace \hspace{0.1cm} $ is the set of the linear regression models that are the result of adding one predictor to the model $M_j$

### Forward Selection in `Python` <a class="anchor" id="24"></a>

We are going to develop the Forward algorithm in `Python` :

In [325]:
def forward ( X, y, varchar_process="dummy_dropfirst", criteria="AIC" ):

        X = varcharProcessing(X , varchar_process = varchar_process)

        cols = X.columns.tolist()

        regressor = sm.OLS(y, X).fit()

        selected_cols = ["intercept"]

        other_cols = cols.copy()
        other_cols.remove("intercept")

        
        model = sm.OLS(y, X[selected_cols]).fit() 

        train_errors_MSE = [ ( (y - model.predict(X[ selected_cols ]))**2 ).mean()  ]

        Models = pd.DataFrame([[ selected_cols[0], round(train_errors_MSE[0]) , model.aic, model.bic, round(model.rsquared_adj, 4) ]], columns = ["model","train_error_MSE","AIC","BIC","Adj.R2"])
        
  
############################################################################################################################################################################ 
     
        for i in range(X.shape[1] - 1):

                train_errors = pd.DataFrame(columns = ["Cols", "train_error_MSE"])

                for j in other_cols:

                        model = sm.OLS(y, X[ selected_cols + [j] ] ).fit()

                        train_error_MSE = ( (y - model.predict(X[ selected_cols + [j] ]))**2 ).mean()
                                              
                        train_errors = pd.concat( [train_errors, pd.DataFrame([[ j , train_error_MSE ]], columns = ["Cols","train_error_MSE"] ) ] )

                train_errors = train_errors.sort_values(by=["train_error_MSE"]).reset_index(drop=True)
                
                model = sm.OLS(y, X[ selected_cols + [train_errors["Cols"][0]] ]).fit()  

                train_error_MSE = ( (y - model.predict(X[ selected_cols + [train_errors["Cols"][0]] ]))**2 ).mean()

                train_errors_MSE.append( train_error_MSE )
                              
                selected_cols.append( train_errors["Cols"][0] )
                other_cols.remove( train_errors["Cols"][0] )

                Models = pd.concat([Models, pd.DataFrame([[ selected_cols[0:(i+2)] , round(train_errors_MSE[i+1]), model.aic, model.bic, round(model.rsquared_adj,5) ]], columns = ["model", "train_error_MSE", "AIC","BIC", "Adj.R2"]) ])        
                

 ############################################################################################################################################################################
              
        Models.index = range(0, len(Models))

        if (criteria == "AIC"):

                Models = Models.sort_values(by=["AIC"]).reset_index(drop=False)

                Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 
        
        elif (criteria == "BIC"):

                Models = Models.sort_values(by=["BIC"]).reset_index(drop=False)

                Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 

        elif (criteria == "Cp"):

                 Models = Models.sort_values(by=["Cp"]).reset_index(drop=False)

                 Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 

        elif (criteria == "Adj.R2"):

                  Models = Models.sort_values(by=["Adj.R2"], ascending=False).reset_index(drop=False)
                  
                  Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 

        elif (criteria == "train_error_MSE"):

                  Models = Models.sort_values(by=["train_error_MSE"]).reset_index(drop=False)
                  
                  Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 
        
        elif ( criteria == "none"):

                  Final_Model = "not compute, use some criteria (AIC, BIC, Adj.R2 or train_error_MSE)"


               
        return Models , Final_Model

In [326]:
Models , Final_Model = forward(X, y, criteria="AIC")

In [327]:
Models

,index,model,train_error_MSE,AIC,BIC,Adj.R2
0,5,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2568592674038,59852.348316,59885.661739,0.69638
1,6,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2566131657360,59852.522224,59891.387885,0.69652
2,7,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2563718320541,59852.729811,59897.147709,0.69664
3,4,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2572226401931,59853.041370,59880.802556,0.69611
4,8,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2562772731258,59854.027050,59903.997186,0.69659
5,3,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2579488352477,59856.412019,59878.620968,0.69542
6,2,"[intercept, size_in_m_2, no_of_bedrooms]",2666941699634,59917.927231,59934.583943,0.68526
7,1,"[intercept, size_in_m_2]",2936347992629,60099.253467,60110.357942,0.65364
8,0,intercept,8482279037222,62118.101353,62123.653590,-0.00000


In [328]:
from IPython.display import HTML 

HTML(Models.to_html(classes='table table-striped'))

,index,model,train_error_MSE,AIC,BIC,Adj.R2
0,5,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2]",2568592674038,59852.348316,59885.661739,0.69638
1,6,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1]",2566131657360,59852.522224,59891.387885,0.69652
2,7,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1, quality_3]",2563718320541,59852.729811,59897.147709,0.69664
3,4,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude]",2572226401931,59853.041370,59880.802556,0.69611
4,8,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1, quality_3, no_of_bathrooms]",2562772731258,59854.027050,59903.997186,0.69659
5,3,"[intercept, size_in_m_2, no_of_bedrooms, latitude]",2579488352477,59856.412019,59878.620968,0.69542
6,2,"[intercept, size_in_m_2, no_of_bedrooms]",2666941699634,59917.927231,59934.583943,0.68526
7,1,"[intercept, size_in_m_2]",2936347992629,60099.253467,60110.357942,0.65364
8,0,intercept,8482279037222,62118.101353,62123.653590,-0.00000


So, the linear regression model selected with ***forward*** algorithm and the $AIC$ criteria is saved in  `Final_Model`, and it is the same model that is selected with ***best subset*** algorithm  :

$$ price = intercept + size\_in\_m\_2 + longitude + latitude + no\_of\_bedrooms + quality\_2 $$

### Forward Selection in `R` <a class="anchor" id="25"></a>

We are going to use the `step` function, with which we can do a forward selection based on $AIC$ criteria in `R`

In [343]:
%%R

model <- step( object = lm(price  ~ size_in_m_2  + longitude  + latitude + no_of_bedrooms  + no_of_bathrooms + quality , data = data_R), direction = "forward")

summary(model)

Start:  AIC=54447.87
price ~ size_in_m_2 + longitude + latitude + no_of_bedrooms + 
    no_of_bathrooms + quality


Call:
lm(formula = price ~ size_in_m_2 + longitude + latitude + no_of_bedrooms + 
    no_of_bathrooms + quality, data = data_R)

Residuals:
      Min        1Q    Median        3Q       Max 
-13398393   -562302     68143    562733  15384235 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -6.207e+07  2.995e+07  -2.073   0.0383 *  
size_in_m_2      3.566e+04  7.238e+02  49.271  < 2e-16 ***
longitude       -1.677e+06  6.908e+05  -2.428   0.0153 *  
latitude         6.115e+06  7.809e+05   7.830 8.03e-15 ***
no_of_bedrooms  -8.367e+05  8.282e+04 -10.102  < 2e-16 ***
no_of_bathrooms -5.712e+04  6.829e+04  -0.836   0.4030    
quality1         1.400e+05  8.358e+04   1.675   0.0940 .  
quality2         3.406e+05  1.551e+05   2.196   0.0282 *  
quality3         2.788e+05  1.976e+05   1.410   0.1586    
---
Signif. codes:  0 '***' 0.001 '**

 
We get different results in R and Python, this could be because the `step` function is using another version of the direct selection algorithm, or something similar. Also, our Python algorithms allow to consider models like price ~ size_in_m_2 + quality_2, while in R this is not possible, in R you have to include all the dummies associated to a categorical variable, so in that case, in R, just you could set price ~ size_in_m_2 + quality ( = price ~ size_in_m_2 + quality_1 + quality_2 + quality_3 in Python)

In this article I have focused mainly on Python, so I recommend use Python algorithms , since they have been developed from zero, taking all the care, and all the procedures have been exposed here, so that everyone can check it out.


Using  `olsrr` package :


In [345]:
%%R 

model <- step( object = lm(price  ~ size_in_m_2  + longitude  + latitude + no_of_bedrooms  + no_of_bathrooms + quality , data = data_R), direction = "forward")

ols_step_forward_p(model)

Start:  AIC=54447.87
price ~ size_in_m_2 + longitude + latitude + no_of_bedrooms + 
    no_of_bathrooms + quality


                                   Selection Summary                                     
----------------------------------------------------------------------------------------
        Variable                        Adj.                                                
Step       Entered        R-Square    R-Square      C(p)         AIC            RMSE        
----------------------------------------------------------------------------------------
   1    size_in_m_2         0.6538      0.6536    271.3798    60101.2535    1714477.7667    
   2    no_of_bedrooms      0.6856      0.6853     74.0667    59919.9272    1634364.7794    
   3    latitude            0.6959      0.6954     11.3666    59858.4120    1607767.3954    
   4    longitude           0.6968      0.6961      7.9940    59855.0414    1605925.0975    
   5    quality             0.6978      0.6966      3.6996

### Backward Selection <a class="anchor" id="26"></a>

Backward selection consist in the following algorithm :

We have $p$ predictors: $X_1,...,X_p$

- We train the full linear regression model $(M_p)$ 
  
- We train all the  linear models that are the result of removing one predictor to the model $M_p$ , and we select one $(M_{p-1})$ under some criteria, for example the one with **less** $train \hspace{0.1cm} error$.


- We train all the linear models that are the result of removing one predictor to the model $M_{p-1}$ , and we select one $(M_{p-2})$ under the same criteria.
  
   $\dots$ 

- We train all the linear models that are the result of removing one predictor to the model $M_{2}$ , and we select one $(M_{1})$ under the same criteria.

- We train the null linear model $(M_0)$


We select one of the models $\hspace{0.1cm}\lbrace M_0,M_1,...,M_{p-1},M_p \rbrace\hspace{0.1cm} $ under some criteria, for example the one with **less** $ \hspace{0.1cm} train\hspace{0.1cm} error$, $AIC$, $BIC$ or $Cp \hspace{0.1cm}$, or **greater**  $\hspace{0.1cm} \widehat{R}^2$  


Scheme of the algorithm:

- $M_p$

- $\lbrace  M_p \hspace{0.1cm} - \text{1 predictor} \rbrace \underset{ \text{train  error} }{\Rightarrow}M_{p-1}$

- $\lbrace  M_{p-1} \hspace{0.1cm} - \text{1 predictor} \rbrace \underset{ \text{train  error} }{\Rightarrow}M_{p-2}$

$\hspace{0.8cm} \dots$

- $\lbrace  M_{2} \hspace{0.1cm} - \text{1 predictor} \rbrace \underset{ \text{train  error} }{\Rightarrow}M_{1}$

- $M_0$

- $\lbrace M_0, M_1 , ..., M_p \rbrace \underset{\text{train error}, AIC, BIC, C_p, \widehat{R}^2 }{\Rightarrow} \hspace{0.1cm} M\hspace{0.05cm}^* \hspace{0.1cm} (Best \hspace{0.1cm} Model) $

### Backward Selection in `Python` <a class="anchor" id="27"></a>

We are going to develop the Backward algorithm in `Python` :

In [330]:
from multiprocessing.resource_sharer import stop


def backward ( X,y, varchar_process="dummy_dropfirst", criteria="AIC"):

        X = varcharProcessing(X , varchar_process = varchar_process)

        cols = X.columns.tolist()

        regressor = sm.OLS(y, X).fit()

        selected_cols = cols.copy()

        other_cols = cols.copy()
        other_cols.remove("intercept")

        predictor_out = [ ]

        predictor_out_reversed = []

        
        model = sm.OLS(y, X[selected_cols]).fit() 

        train_errors_MSE = [ ( (y - model.predict(X[ selected_cols ]))**2 ).mean()  ]

        Models = pd.DataFrame([[ round(train_errors_MSE[0]) , model.aic, model.bic, round(model.rsquared_adj, 4) ]], columns = ["train_error_MSE","AIC","BIC", "Adj.R2"])
        
  
 ######################################################################################  
       
        for i in range(X.shape[1]-1):

                train_errors = pd.DataFrame(columns = ["Cols", "train_error_MSE"])

                for j in other_cols:

                        selected_cols.remove(j)

                        model = sm.OLS(y, X[ selected_cols ] ).fit()

                        train_error_MSE = ( (y - model.predict(X[ selected_cols ]))**2 ).mean()
                                             
                        train_errors = pd.concat( [train_errors, pd.DataFrame([[ j , train_error_MSE ]], columns = ["Cols", "train_error_MSE"] ) ] )

                        selected_cols.append(j)

                train_errors = train_errors.sort_values(by=["train_error_MSE"]).reset_index(drop=True)
                
                selected_cols.remove(train_errors["Cols"][0])

                predictor_out.append(train_errors["Cols"][0])

                model = sm.OLS(y, X[ selected_cols ]).fit()  

                train_error_MSE = ( (y - model.predict(X[ selected_cols ]))**2 ).mean()

                train_errors_MSE.append( train_error_MSE )
                              
                other_cols.remove( train_errors["Cols"][0] )


                Models = pd.concat([Models, pd.DataFrame([[ round(train_errors_MSE[i+1]), model.aic, model.bic, round(model.rsquared_adj,5) ]], columns = ["train_error_MSE", "AIC","BIC", "Adj.R2"]) ])        

##################################################################################################

        Models.index = range(0,X.shape[1])


        idx = len(predictor_out + ['intercept']) - 1
        newList = []
        
        while (idx >= 0):
                predictor_out_reversed.append((predictor_out + ['intercept'])[idx])
                idx = idx - 1


        a = pd.Series([])
        
        for i in range(X.shape[1]):

                a[i] =  predictor_out_reversed[0:(len(cols)-i)] 


        Models = pd.concat([pd.DataFrame({'model': a}) , Models],  axis=1)

################################################################################################


        if (criteria == "AIC"):

                Models = Models.sort_values(by=["AIC"]).reset_index(drop=False)

                Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 
        
        elif (criteria == "BIC"):

                Models = Models.sort_values(by=["BIC"]).reset_index(drop=False)

                Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 

        elif (criteria == "Adj.R2"):

                  Models = Models.sort_values(by=["Adj.R2"], ascending=False).reset_index(drop=False)
                  
                  Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 

        elif (criteria == "train_error_MSE"):

                  Models = Models.sort_values(by=["train_error_MSE"]).reset_index(drop=False)
                  
                  Final_Model = sm.OLS(y, X[ Models["model"][0] ]).fit() 
        
        elif ( criteria == "none"):

                Final_Model = "not compute, use some criteria (AIC, BIC, Adj.R2 or train_error_MSE)"


        return Models , Final_Model

In [331]:
Models , Final_Model = backward ( X , y, varchar_process="dummy_dropfirst", criteria="AIC")

In [332]:
Models

,index,model,train_error_MSE,AIC,BIC,Adj.R2
0,3,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2568592674038,59852.348316,59885.661739,0.69638
1,2,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2566131657360,59852.522224,59891.387885,0.69652
2,1,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2563718320541,59852.729811,59897.147709,0.69664
3,4,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2572226401931,59853.041370,59880.802556,0.69611
4,0,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2562772731258,59854.027050,59903.997186,0.69660
5,5,"[intercept, size_in_m_2, no_of_bedrooms, latit...",2579488352477,59856.412019,59878.620968,0.69542
6,6,"[intercept, size_in_m_2, no_of_bedrooms]",2666941699634,59917.927231,59934.583943,0.68526
7,7,"[intercept, size_in_m_2]",2936347992629,60099.253467,60110.357942,0.65364
8,8,[intercept],8482279037222,62118.101353,62123.653590,-0.00000


In [333]:
from IPython.display import HTML 

HTML(Models.to_html(classes='table table-striped'))

,index,model,train_error_MSE,AIC,BIC,Adj.R2
0,3,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2]",2568592674038,59852.348316,59885.661739,0.69638
1,2,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1]",2566131657360,59852.522224,59891.387885,0.69652
2,1,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1, quality_3]",2563718320541,59852.729811,59897.147709,0.69664
3,4,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude]",2572226401931,59853.041370,59880.802556,0.69611
4,0,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1, quality_3, no_of_bathrooms]",2562772731258,59854.027050,59903.997186,0.69660
5,5,"[intercept, size_in_m_2, no_of_bedrooms, latitude]",2579488352477,59856.412019,59878.620968,0.69542
6,6,"[intercept, size_in_m_2, no_of_bedrooms]",2666941699634,59917.927231,59934.583943,0.68526
7,7,"[intercept, size_in_m_2]",2936347992629,60099.253467,60110.357942,0.65364
8,8,[intercept],8482279037222,62118.101353,62123.653590,-0.00000


So, the linear regression model selected with ***backward*** algorithm and the $AIC$ criteria is saved in  `Final_Model`, and it is the same model that is selected with ***best subset*** and ***forward*** algorithms  :

$$ price = intercept + size\_in\_m\_2 + longitude + latitude + no\_of\_bedrooms + quality\_2 $$

In [335]:
Models , Final_Model = backward ( X , y, varchar_process="dummy_dropfirst", criteria="none")

In [336]:
HTML(Models.to_html(classes='table table-striped'))

,model,train_error_MSE,AIC,BIC,Adj.R2
0,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1, quality_3, no_of_bathrooms]",2562772731258,59854.027050,59903.997186,0.69660
1,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1, quality_3]",2563718320541,59852.729811,59897.147709,0.69664
2,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2, quality_1]",2566131657360,59852.522224,59891.387885,0.69652
3,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude, quality_2]",2568592674038,59852.348316,59885.661739,0.69638
4,"[intercept, size_in_m_2, no_of_bedrooms, latitude, longitude]",2572226401931,59853.041370,59880.802556,0.69611
5,"[intercept, size_in_m_2, no_of_bedrooms, latitude]",2579488352477,59856.412019,59878.620968,0.69542
6,"[intercept, size_in_m_2, no_of_bedrooms]",2666941699634,59917.927231,59934.583943,0.68526
7,"[intercept, size_in_m_2]",2936347992629,60099.253467,60110.357942,0.65364
8,[intercept],8482279037222,62118.101353,62123.653590,-0.00000


In [337]:
Final_Model

'not compute, use some criteria (AIC, BIC, Adj.R2 or train_error_MSE)'

### Backward Selection in `R` <a class="anchor" id="28"></a>

We are going to use the `step` function, with which we can do a backward selection based on $AIC$ criteria in `R`

In [341]:
%%R

Modelo_Backward <- step( object = lm(price  ~ size_in_m_2  + longitude  + latitude + no_of_bedrooms  + no_of_bathrooms + quality , data = data_R), direction = "backward")

summary(Modelo_Backward)

Start:  AIC=54447.87
price ~ size_in_m_2 + longitude + latitude + no_of_bedrooms + 
    no_of_bathrooms + quality

                  Df  Sum of Sq        RSS   AIC
- no_of_bathrooms  1 1.8013e+12 4.8839e+15 54447
<none>                          4.8821e+15 54448
- quality          3 1.6585e+13 4.8987e+15 54448
- longitude        1 1.5178e+13 4.8973e+15 54452
- latitude         1 1.5788e+14 5.0400e+15 54507
- no_of_bedrooms   1 2.6279e+14 5.1449e+15 54546
- size_in_m_2      1 6.2511e+15 1.1133e+16 56016

Step:  AIC=54446.57
price ~ size_in_m_2 + longitude + latitude + no_of_bedrooms + 
    quality

                 Df  Sum of Sq        RSS   AIC
<none>                         4.8839e+15 54447
- quality         3 1.6208e+13 4.9001e+15 54447
- longitude       1 1.5065e+13 4.8989e+15 54450
- latitude        1 1.5635e+14 5.0402e+15 54505
- no_of_bedrooms  1 5.3313e+14 5.4170e+15 54642
- size_in_m_2     1 6.4504e+15 1.1334e+16 56048

Call:
lm(formula = price ~ size_in_m_2 + longitude + latitu

Using  `olsrr` package :


In [346]:
%%R 

model <- step( object = lm(price  ~ size_in_m_2  + longitude  + latitude + no_of_bedrooms  + no_of_bathrooms + quality , data = data_R), direction = "forward")

ols_step_backward_p(model)

Start:  AIC=54447.87
price ~ size_in_m_2 + longitude + latitude + no_of_bedrooms + 
    no_of_bathrooms + quality



                                  Elimination Summary                                   
---------------------------------------------------------------------------------------
        Variable                         Adj.                                              
Step        Removed        R-Square    R-Square     C(p)        AIC            RMSE        
---------------------------------------------------------------------------------------
   1    no_of_bathrooms      0.6978      0.6966    3.6996    59854.7298    1604534.1988    
---------------------------------------------------------------------------------------


## Bibliography

https://nathancarter.github.io/how2data/site/

https://github.com/talhahascelik/python_stepwiseSelection/blob/master/stepwiseSelection.py

http://www.science.smith.edu/~jcrouser/SDS293/labs/lab8-py.html

https://statweb.stanford.edu/~jtaylo/courses/stats203/notes/selection.pdf

https://xavierbourretsicotte.github.io/subset_selection.html#Comparing-models:-AIC,-BIC,-Mallows'CP

Linear Models with R. Julian Faraway.

Notes from professor Sandra Benitez Peña from UC3M